This notebook explores the On-Street Curb Management dataset compiled from four NYC data sources (curbs, parking meters, loading zones, and truck routes)

In [30]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.wkt import loads
from IPython.display import display

In [40]:
# Load the final merged dataset
df = pd.read_csv("data/on_street_curb_management.csv")

# Preview data
print(f"🔢 Total rows: {len(df)}")
df.head()

/var/folders/0q/f6wdygs95dz97f63ll8xv7mc0000gn/T/ipykernel_61619/2306712696.py:2: DtypeWarning: Columns (7,8,9,10,11,12,13,15,17,18,19,20,21,22,25,27,28,29,38,58,61,66,67,68) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/on_street_curb_management.csv")


🔢 Total rows: 221289


,SOURCE_ID,FEAT_CODE,SUB_CODE,STATUS,SHAPE_Leng,geometry,dist_to_loading,loading_order_number,loading_record_type,loading_order_type,...,truck_localtunl,truck_localbrg,truck_thrutunl,truck_thrubrg,truck_assem_dist,truck_cong_dist,truck_coun_dist,truck_st_sen_dis,truck_thruexwy,truck_borocd
0,20225003296,2250,225000,Unchanged,40.459092,MULTILINESTRING ((-73.89937077176755 40.658986...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20225000231,2250,225000,Unchanged,169.098569,MULTILINESTRING ((-73.90416765196942 40.679178...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20225001187,2250,225000,Unchanged,890.605868,MULTILINESTRING ((-73.92525824301764 40.643611...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20225003031,2250,225000,Unchanged,703.147796,MULTILINESTRING ((-73.90249394429374 40.646745...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20225001161,2250,225000,Unchanged,1003.955771,MULTILINESTRING ((-73.9345590345314 40.6581315...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# Summarize key fields
df[['loading_order_number', 'meter_Meter Number', 'truck_street']].notnull().sum()

loading_order_number      474
meter_Meter Number      14909
truck_street            20361
dtype: int64

In [42]:
# Load and parse geometries
df = pd.read_csv("data/on_street_curb_management.csv")

# Ensure geometry is parsed
df['geometry'] = df['geometry'].apply(loads)
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

# Quick preview
display(gdf.head())

/var/folders/0q/f6wdygs95dz97f63ll8xv7mc0000gn/T/ipykernel_61619/1727622264.py:2: DtypeWarning: Columns (7,8,9,10,11,12,13,15,17,18,19,20,21,22,25,27,28,29,38,58,61,66,67,68) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/on_street_curb_management.csv")


,SOURCE_ID,FEAT_CODE,SUB_CODE,STATUS,SHAPE_Leng,geometry,dist_to_loading,loading_order_number,loading_record_type,loading_order_type,...,truck_localtunl,truck_localbrg,truck_thrutunl,truck_thrubrg,truck_assem_dist,truck_cong_dist,truck_coun_dist,truck_st_sen_dis,truck_thruexwy,truck_borocd
0,20225003296,2250,225000,Unchanged,40.459092,"MULTILINESTRING ((-73.89937 40.65899, -73.8993...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20225000231,2250,225000,Unchanged,169.098569,"MULTILINESTRING ((-73.90417 40.67918, -73.9041...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20225001187,2250,225000,Unchanged,890.605868,"MULTILINESTRING ((-73.92526 40.64361, -73.9253...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20225003031,2250,225000,Unchanged,703.147796,"MULTILINESTRING ((-73.90249 40.64675, -73.9025...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20225001161,2250,225000,Unchanged,1003.955771,"MULTILINESTRING ((-73.93456 40.65813, -73.9346...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Create a folium map centered in NYC
#uncomment line below to run
m = folium.Map(location=[40.7128, -74.0060], zoom_start=12)

# Helper to add points
def add_markers(gdf, col, color, label):
    for _, row in gdf[gdf[col].notnull()].iterrows():
        folium.CircleMarker(
            location=[row.geometry.centroid.y, row.geometry.centroid.x],
            radius=3,
            color=color,
            fill=True,
            fill_opacity=0.6,
            popup=label
        ).add_to(m)

# Add loading zones, meters, and truck indicators
add_markers(gdf, 'loading_order_number', 'green', 'Loading Zone')
add_markers(gdf, 'meter_Meter Number', 'blue', 'Parking Meter')
add_markers(gdf, 'truck_street', 'red', 'Truck Route')

# Display in notebook
m

m.save('folium_map.html')  # Save map externally to avoid heavy output